In [3]:
!pip install pypdf
!pip install langchain-huggingface
!pip install faiss-cpu
!pip install torch
%load_ext autotime

time: 356 µs (started: 2024-10-21 00:43:05 +00:00)


In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings

time: 4.11 s (started: 2024-10-21 00:43:23 +00:00)


In [6]:
# Load the documents
file_path = ("SejalUpadhyeResume.pdf")
loader = PyPDFLoader(file_path)
documents=loader.load()

time: 101 ms (started: 2024-10-21 00:43:30 +00:00)


In [7]:
# Split the documents
text_splitter=RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=64)
docs=text_splitter.split_documents(documents)

time: 734 µs (started: 2024-10-21 00:43:32 +00:00)


In [13]:
# Embed and store the documents chunks
db = FAISS.from_documents(docs, HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))

/tmp/ipykernel_4823/1083491850.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  db = FAISS.from_documents(docs, HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))
/usr/local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


time: 3.86 s (started: 2024-10-21 00:44:42 +00:00)


In [14]:
# Define a retriever 
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 5})

time: 609 µs (started: 2024-10-21 00:44:49 +00:00)


In [15]:
# model_name = "THUDM/glm-4-9b-chat"
# model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code = True)
# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)

tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

time: 2.55 s (started: 2024-10-21 00:44:51 +00:00)


In [16]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)


llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """You are an expert assistant specialized in question answering. Use the provided context to answer the user's question accurately and concisely. 

Context:
{context}

Question:
{question}

Answer:
"""

# prompt_template = """
# <|system|>
# You are an expert assistant specialized in question answering. Use the provided context to answer the user's question accurately and concisely. 
# Context:
# {context}

# </s>
# <|user|>
# {question}
# </s>
# <|assistant|>

#  """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

time: 33.1 ms (started: 2024-10-21 00:44:55 +00:00)


/tmp/ipykernel_4823/1152053709.py:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [17]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain

time: 578 µs (started: 2024-10-21 00:44:58 +00:00)


In [18]:
print(rag_chain.invoke("what is the name of the candidate?"))

You are an expert assistant specialized in question answering. Use the provided context to answer the user's question accurately and concisely. 

Context:
[Document(metadata={'source': 'SejalUpadhyeResume.pdf', 'page': 0}, page_content='solving the problem of Structure-aware Language Modelling\nM.TECH COURSE PROJECTS\n•Question Answering System\n(CS 772: Deep Learning for Natural Language Processing, Instructor: Prof. Pushpak Bhattacharyya ) (Jan’22- May’22)\n◦Used only factoid type questions from the SQuAD dataset\n◦Implemented an encoder and decoder network based on RNN\n•Image Caption Validation\n(CS 772: Deep Learning for Natural Language Processing, Instructor: Prof. Pushpak Bhattacharyya ) (Jan’22- May’22)'), Document(metadata={'source': 'SejalUpadhyeResume.pdf', 'page': 1}, page_content='•Tools & Libraries : Tensorflow, PyTorch, L ATEX, Git, MySQL\nACHIEVEMENTS AND EXTRA CURRICULAR ACTIVITIES\n• Secured AIR-17 in GATE 2021\n•CSE Department Badminton Girls Winner at IIT Bombay\n•